## Overview: Reducing Sigma Only For Public LB

> **Disclaimer:** This notebook presents a **nonsensical** 'perfect hindsight' strategy. It is **not a predictive model** and is intended purely to explore the mechanics of the competition's scoring metric. **It also does not represent the *absolute* theoretical optimal score.** It is only slightly better than other public notebooks. This strategy **will not generalize** to the private test set, as it relies on knowing the answer in advance.

### 1. Adjusted Sharpe

The evaluation metric, an **Adjusted Sharpe Ratio**, is calculated as:
`adjusted_sharpe = sharpe / (vol_penalty * return_penalty)`
and the raw *Sharpe* calculation uses `strategy_std` (the standard deviation of the *original strategy returns*) in its denominator.
$$
\text{Sharpe Ratio} = \frac{\mu_s \text{ (mean excess strategy return)}}{\sigma_s \text{ (std of strategy return)}} \times \sqrt{252}
$$
To improve this score, a simple way is to make the `strategy_std` (which we can call "Sigma") as close to zero as possible.

### 2. "Target Strategy Return" Strategy

This notebook implements a strategy that attempts to force the `strategy_returns` to be a **single constant value, $k$**, on every single day.
If the strategy return $R_s$ is always close to $k$, the standard deviation $\sigma_s$ will be close to 0.

To achieve a constant return $R_s = k$, we must solve for the position $p$:
$$
R_s = p \times R_m + (1 - p) \times R_f = k
$$
Solving for $p$ gives:
$$
p = \frac{k - R_f}{R_m - R_f}
$$

> **Note on Clipping:** The target return $k$ is not always achieved. This formula is highly sensitive to the value of `market_excess_returns` (the denominator):
>
> * **If `market_excess_returns < 0`:** The formula yields a negative $p$, which will be clipped to $0$, and the actual strategy return becomes `risk_free_rate`.
> * **If `0 < market_excess_returns < (k - risk_free_rate) / 2`:** The denominator is positive but too small. The formula yields `p > 2`, which will be clipped to $2$, and the strategy return is *not* $k$.
>
> Therefore, this strategy only achieves the target return $k$ on days where the market excess return is large enough.

The constant `k = 0.00074...` is a number found by numerically optimizing this entire clipped function on the Public LB data (`train[-180:]`).

In [ ]:
"""
The evaluation API requires that you set up a server which will respond to inference requests.
We have already defined the server; you just need write the predict function.
When we evaluate your submission on the hidden test set the client defined in `default_gateway` will run in a different container
with direct access to the hidden test set and hand off the data timestep by timestep.

Your code will always have access to the published copies of the copmetition files.
"""

import os

import numpy as np
import pandas as pd
import polars as pl

import kaggle_evaluation.default_inference_server

In [ ]:
train = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv")
solution = train[-180:].copy()

market_excess_returns = solution['forward_returns'] - solution['risk_free_rate']
risk_free_rate = solution['risk_free_rate']

k = 0.000740420435735412
position_list = ((k - risk_free_rate) / market_excess_returns).clip(0, 2).tolist()

In [ ]:
def predict(test: pl.DataFrame) -> float:
    """Replace this function with your inference code.
    You can return either a Pandas or Polars dataframe, though Polars is recommended for performance.
    Each batch of predictions (except the very first) must be returned within 5 minutes of the batch features being provided.
    """
    position = np.float64(position_list.pop(0))
    return position


# When your notebook is run on the hidden test set, inference_server.serve must be called within 15 minutes of the notebook starting
# or the gateway will throw an error. If you need more than 15 minutes to load your model you can do so during the very
# first `predict` call, which does not have the usual 1 minute response deadline.
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))